# Gradiente Descendente - Atv. 2

## Configurações e Classes Utilitárias

In [22]:
#%pip install numpy matplotlib IPython
#%matplotlib inline

Utilizaremos pra essa atividade alguns frameworks, os quais serão configurados conforme o código abaixo.

In [23]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

### Classe SSR

Definiremos a seguir a classe SSR (Sum of Squared Residuals), que agrupa os métodos de cálculo de gradiente, e de custo implementando e seguindo as seguintes fórmulas:

- Custo(SSR): $ \ J(m,b) =\sum^n_{i=1}(y_i-(mx_i+b))^2 $

- Gradiente de $ b \ $: $\ \dfrac{\partial J}{\partial b}=-2\sum^n_{i=1}(y_i-(mx_i+b)) $

- Gradiente de $ m \ $: $\ \dfrac{\partial J}{\partial m}=-2\sum^n_{i=1}x_i(y_i-(mx_i+b)) $

In [24]:
class SSR:
    def calculate_cost(X: np.array, y: np.array, m, b):
        y_pred = m * X + b
        residuals = y - y_pred
        return np.sum(residuals**2)

    def calculate_gradient_b(X, y, m, b):
        y_pred = m * X + b
        return -2 * np.sum(y - y_pred)

    def calculate_gradient_m(X, y, m, b):
        y_pred = m * X + b
        return -2 * np.sum(X * (y - y_pred))

### Animar com Slope Fixo

A classe FixedSlopePlot, tem como objetivo abstrair o processo de plotagem de gráficos, bem como a formatação do gráfico.\
Para o caso de Slope Fixado, a plotagem consiste em dois gráficos lado a lado, onde o gráfico do lado esquerdo mostra a curva de minimização de resíduo, e do lado direito, mostra a reta de ajuste ao dataset.

In [25]:
class FixedSlopePlot:
    def __init__(self, X, y, fixed_slope):
        self.X = X
        self.y = y
        self.fixed_slope = fixed_slope
        self.fig, (self.ax_left, self.ax_right) = plt.subplots(1, 2, figsize=(14, 6))

        self.point_cost, = self.ax_left.plot([], [], 'ro', markersize=10, zorder=5)
        self.line_tangent, = self.ax_left.plot([], [], 'g-', linewidth=2.5, label='Tangente')
        self.line_reg, = self.ax_right.plot([], [], 'r-', linewidth=3)

    def setup_background(self, history):
        # curva de custo
        b_values = [h[0] for h in history]
        buffer = 1.5
        b_range = np.linspace(min(b_values)-buffer, max(b_values)+buffer, 100)
        cost_curve = [SSR.calculate_cost(self.X, self.y, self.fixed_slope, b) for b in b_range]

        self.ax_left.plot(b_range, cost_curve, 'k--', alpha=0.4)
        self.ax_left.set_title("Minimização do Custo (Visualizando a Derivada)")
        self.ax_left.set_xlabel("Intercepto (b)")
        self.ax_left.set_ylabel("Custo (SSR)")
        self.ax_left.grid(True, linestyle=':', alpha=0.6)

        # reta de ajuste
        self.ax_right.scatter(self.X, self.y, color='blue', s=100)
        self.ax_right.set_xlim(min(self.X)-1, max(self.X)+1)
        self.ax_right.set_ylim(min(self.y)-1, max(self.y)+1)
        self.ax_right.set_title(f"Ajuste da Reta (Slope fixo: {self.fixed_slope})")
        self.ax_right.grid(True, linestyle=':', alpha=0.6)

    def create_animation(self, history):
        self.setup_background(history)

        def update(frame):
            b, cost, grad = history[frame]
            self.point_cost.set_data([b], [cost])

            # reta tangente
            x_tan = np.linspace(b - 1.5, b + 1.5, 10)
            y_tan = cost + grad * (x_tan - b)
            self.line_tangent.set_data(x_tan, y_tan)

            # atualiza reta de ajuste
            x_line = np.array([self.ax_right.get_xlim()[0], self.ax_right.get_xlim()[1]])
            y_line = self.fixed_slope * x_line + b
            self.line_reg.set_data(x_line, y_line)

            self.ax_right.legend([f"Intercept = {b:.4f}"], loc='upper left')

            return self.point_cost, self.line_tangent, self.line_reg

        plt.close()
        return FuncAnimation(self.fig, update, frames=len(history), interval=200, blit=True)

### Animar Regressão Completa

Podemos observar que ao otimizarmos dois parâmetros simultaneamente, o gráfico de minimização seria tridimensional, o que nesse contexto não é de importância, então a esta classe apenas mostra a animação centralizada da reta se ajustando melhor a cada iteração, ao dados.

In [26]:
class CompleteRegressionPlot:
    def __init__(self, X, y):
        self.X = X
        self.y = y
        self.fig, self.ax = plt.subplots(figsize=(8, 6))

        # Elementos Gráficos
        self.line_reg, = self.ax.plot([], [], 'r-', linewidth=3)
        self.scatter = self.ax.scatter(self.X, self.y, color='blue', s=100, label='Dados')

    def setup_background(self):
        self.ax.set_xlim(min(self.X)-1, max(self.X)+1)
        self.ax.set_ylim(min(self.y)-1, max(self.y)+1)
        self.ax.set_title("Regressão Completa (Otimizando m e b)")
        self.ax.grid(True, linestyle=':', alpha=0.6)

    def create_animation(self, history):
        self.setup_background()

        def update(frame):
            # Desempacota a tupla completa de 5 elementos
            m, b, cost, grad_m, grad_b = history[frame]

            # Atualiza Reta
            x_line = np.array([self.ax.get_xlim()[0], self.ax.get_xlim()[1]])
            y_line = m * x_line + b
            self.line_reg.set_data(x_line, y_line)

            # Atualiza Legenda
            self.ax.legend([f"Slope (m) = {m:.4f}\nIntercept (b) = {b:.4f}"], loc='upper left')

            return self.line_reg,

        plt.close()
        return FuncAnimation(self.fig, update, frames=len(history), interval=100, blit=True)

## Item A - Regressão Linear com Slope Fixado

Conforme o comando da atividade, para este exercício proposto, focaremos na otimzação apenas do intercept da função que se ajusta ao nosso da dataset.\
Reta essa que pode ser definida por: $ p(x)=mx+b$, onde:
- $p(x)$ é o valor previsto pra entrada $x$
- $ m $ é o slope
- $ b $ é o intercept

Nesse caso específico, o valor do slope já é conhecido, que equivale a 0.64, baseado nisso, a regressão deve seguir uma estrutura como:
- $p(x)=0.64x+b$\
onde $b$ seria o único parâmetro a ser otimizado.

In [40]:
def train_fixed_slope(X, y, fixed_slope, start_b, learning_rate=0.01, epochs=30):
    current_b = start_b
    history = []

    for _ in range(epochs):
        cost = SSR.calculate_cost(X, y, fixed_slope, current_b)
        grad = SSR.calculate_gradient_b(X, y, fixed_slope, current_b)

        history.append((current_b, cost, grad))

        current_b = current_b - (learning_rate * grad)

        # quebra se step size == 0
        if (learning_rate * grad) == 0:
            break

        step_size = learning_rate * grad
        old_b = current_b + step_size
        new_b = current_b
        print(f"| step_size={step_size:.6f} | old_intercept={old_b:.6f} | new_intercept={new_b:.6f} |")

    return history

In [ ]:
# dataset
X_data = np.array([0.75, 2.5, 4.0 , 5])
y_data = np.array([1.5, 2.0, 4.0 , 4.5])

hist_fixed = train_fixed_slope(X_data, y_data, fixed_slope=0.64, start_b=0.0, learning_rate=0.01, epochs=50)

plot_A = FixedSlopePlot(X_data, y_data, fixed_slope=0.64)
anim_A = plot_A.create_animation(hist_fixed)
display(HTML(anim_A.to_jshtml()))